In [5]:
#Feature Engineering
#creo nuevas variables dentro del mismo mes
#Condimentar a gusto con nuevas variables

#limpio la memoria
rm( list=ls() )
gc()

require("data.table")

#Establezco el Working Directory
setwd("/home/devsar/Documents/Austral/Laboratorio de Implementación")


EnriquecerDataset <- function( dataset , arch_destino )
{
  columnas_originales <-  copy(colnames( dataset ))

  #INICIO de la seccion donde se deben hacer cambios con variables nuevas
  #se crean los nuevos campos para MasterCard  y Visa, teniendo en cuenta los NA's
  #varias formas de combinar Visa_status y Master_status
  dataset[ , mv_status01       := pmax( Master_status,  Visa_status, na.rm = TRUE) ]
  dataset[ , mv_status02       := Master_status +  Visa_status ]
  dataset[ , mv_status03       := pmax( ifelse( is.na(Master_status), 10, Master_status) , ifelse( is.na(Visa_status), 10, Visa_status) ) ]
  dataset[ , mv_status04       := ifelse( is.na(Master_status), 10, Master_status)  +  ifelse( is.na(Visa_status), 10, Visa_status)  ]
  dataset[ , mv_status05       := ifelse( is.na(Master_status), 10, Master_status)  +  100*ifelse( is.na(Visa_status), 10, Visa_status)  ]

  dataset[ , mv_status06       := ifelse( is.na(Visa_status), 
                                          ifelse( is.na(Master_status), 10, Master_status), 
                                          Visa_status)  ]

  dataset[ , mv_status07       := ifelse( is.na(Master_status), 
                                          ifelse( is.na(Visa_status), 10, Visa_status), 
                                          Master_status)  ]


    
  #combino MasterCard y Visa
  dataset[ , mv_mfinanciacion_limite := rowSums( cbind( Master_mfinanciacion_limite,  Visa_mfinanciacion_limite) , na.rm=TRUE ) ]

  dataset[ , mv_Fvencimiento         := pmin( Master_Fvencimiento, Visa_Fvencimiento, na.rm = TRUE) ]
  dataset[ , mv_Finiciomora          := pmin( Master_Finiciomora, Visa_Finiciomora, na.rm = TRUE) ]
  dataset[ , mv_msaldototal          := rowSums( cbind( Master_msaldototal,  Visa_msaldototal) , na.rm=TRUE ) ]
  dataset[ , mv_mconsumospesos       := rowSums( cbind( Master_mconsumospesos,  Visa_mconsumospesos) , na.rm=TRUE ) ]
  dataset[ , mv_mconsumosdolares     := rowSums( cbind( Master_mconsumosdolares,  Visa_mconsumosdolares) , na.rm=TRUE ) ]
  dataset[ , mv_mlimitecompra        := rowSums( cbind( Master_mlimitecompra,  Visa_mlimitecompra) , na.rm=TRUE ) ]
  dataset[ , mv_mpagado              := rowSums( cbind( Master_mpagado,  Visa_mpagado) , na.rm=TRUE ) ]
  dataset[ , mv_cconsumos            := rowSums( cbind( Master_cconsumos,  Visa_cconsumos) , na.rm=TRUE ) ]
  dataset[ , mv_mpagominimo          := rowSums( cbind( Master_mpagominimo,  Visa_mpagominimo) , na.rm=TRUE ) ]
  dataset[ , mv_mpagosdolares          := rowSums( cbind( Master_mpagosdolares,  Visa_mpagosdolares ) , na.rm=TRUE ) ]
  dataset[ , mv_msaldodolares          := rowSums( cbind( Master_msaldodolares,  Visa_msaldodolares ) , na.rm=TRUE ) ]
  dataset[ , mv_FechaAlta          := pmin( Master_fechaalta,  Visa_fechaalta , na.rm=TRUE ) ]

  dataset[, g_prom_90_mayor_prom_30 := ifelse(
    (ctrx_quarter / 90) > 
    ((catm_trx_other+ctarjeta_debito_transacciones+cforex_buy+cforex_sell+cpagodeservicios + ctransferencias_emitidas + cextraccion_autoservicio + catm_trx)/30),
    TRUE,
    FALSE)]
  dataset[, g_ratio_90_30 := (ctrx_quarter / 90) / ((catm_trx_other+ctarjeta_debito_transacciones+cforex_buy+cforex_sell+cpagodeservicios + ctransferencias_emitidas + cextraccion_autoservicio + catm_trx)/30)]
  dataset[, g_esta_complicado := ifelse(rowSums( cbind(Master_delinquency,  Visa_delinquency, Master_status, Visa_status, na.rm = TRUE)) > 0, TRUE, FALSE )]
  dataset[, g_saldo_total := rowSums( cbind( mcuentas_saldo,  Master_msaldototal, Visa_msaldototal) , na.rm=TRUE ) ]
  dataset[, g_ctxs_virtuales := rowSums( cbind( chomebanking_transacciones,  cmobile_app_trx) , na.rm=TRUE ) ]
  dataset[, g_ctxs_remotas := rowSums( cbind( chomebanking_transacciones,  cmobile_app_trx, ccallcenter_transacciones) , na.rm=TRUE ) ]
  dataset[, g_prop_txs_app :=  ctrx_quarter / g_ctxs_virtuales]
  dataset[, g_es_activo_virtual := ifelse(g_ctxs_virtuales > 5, TRUE, FALSE)]
  dataset[, g_es_activo_remoto := ifelse(g_ctxs_remotas > 5, TRUE, FALSE)]
  dataset[, g_usa_pago_mis_cuentas := ifelse(cpagomiscuentas > 0, TRUE, FALSE)]
  dataset[, g_promedio_mensual_quarter := ctrx_quarter/3]
  dataset[, g_cobra_salario := rowSums( cbind( mpayroll,  mpayroll2) , na.rm=TRUE ) ]
  dataset[, g_tiene_prestamos := rowSums( cbind( mpayroll,  mpayroll2) , na.rm=TRUE ) ]
  dataset[, g_tiene_inversiones := ifelse(rowSums( cbind( cinversion1,  cinversion2) , na.rm=TRUE ) > 0, TRUE, FALSE) ]
  dataset[, g_posee_prestamos := ifelse(rowSums( cbind( cprestamos_personales,  cprestamos_prendarios, cprestamos_hipotecarios) , na.rm=TRUE ) > 0, TRUE, FALSE) ]
  dataset[, g_tiene_saldos_negativos := ifelse(pmin(mcaja_ahorro, mcaja_ahorro_adicional, mcaja_ahorro_dolares, mcuenta_corriente, mcuenta_corriente_adicional, na.rm=TRUE) < 0, TRUE, FALSE)]

  columnas_extendidas <-  copy( setdiff(  colnames(dataset), columnas_originales ) )

  #grabo con nombre extendido
  fwrite( dataset,
          file=arch_destino,
          sep= "," )
}
#------------------------------------------------------------------------------

#kcarpeta_datasets    <- "../input/laboratorio-de-implementacion-i-2021/"   #KAGGLE
kcarpeta_datasets    <- "./datasets/"                          #VM o Ubuntu

#Archivo con datos etiquetados para entrenamiento
karchivo_entrada      <-  paste0(kcarpeta_datasets, "competencia1_2022.csv")
karchivo_salida      <-  paste0(kcarpeta_datasets, "competencia1_2022_FE_4.csv")



#lectura rapida del dataset  usando fread  de la libreria  data.table
dataset1  <- fread(karchivo_entrada)

EnriquecerDataset( dataset1, karchivo_salida )

quit( save="no")


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,683332,36.5,1335898,71.4,1335898,71.4
Vcells,1369447,10.5,52353722,399.5,70711694,539.5
